In [1]:
# Lasciate questa cella così ogni volta che aggiornate un file python, 
# anche gli import nel notebook vengono aggiornati

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from Utils import *
from constants import *

from Environment import *
from Non_Stationary_Environment import *

from Social_influence import *
from Network import Network

from Simulation import *

from Ecommerce import *
from Ecommerce3 import *
from Ecommerce4 import *
from Ecommerce5 import *
from Ecommerce6 import *
from step7.Ecommerce7 import*

## Step 2

In [3]:
simulate_step2()

optimal allocation is: [60. 60. 20. 60.  0.] with a reward of: 457
estimated_opt_allocation is: [60. 60. 20. 60.  0.] with a reward of: 457


## Step 3

In [ ]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step3()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 4

In [11]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step4()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 5

In [ ]:
gpucb_rewards_per_experiment, gpts_rewards_per_experiment, opts = simulate_step5()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 6

In [ ]:
swucb_rewards_per_experiment, cducb_rewards_per_experiment, opts = simulate_step6()
plot_regrets(swucb_rewards_per_experiment, cducb_rewards_per_experiment, opts, ["SWUCB", "CDUCB"])

## Step 7

In [13]:
import numpy as np

from Utils import *
from constants import *

from Environment import *
from Non_Stationary_Environment import *

from Social_influence import *
from Network import Network

from Simulation import *

from Ecommerce import *
from Ecommerce3 import *
from Ecommerce4 import *
from Ecommerce5 import *
from Ecommerce6 import *
from step7.Ecommerce7 import*

In [14]:
env, observations_probabilities, click_probabilities, product_prices,\
                users_reservation_prices, users_poisson_parameters = generate_new_environment()


nodes_activation_probabilities, num_sold_items = estimate_nodes_activation_probabilities(
        click_probabilities,
        users_reservation_prices,
        users_poisson_parameters,
        product_prices,
        observations_probabilities
    )

exp_clicks = env.estimate_num_of_clicks(budgets/B_cap)

ecomm7 = Ecommerce7(B_cap, budgets, product_prices, features, 'TS')


In [16]:
features = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 0}
split_time = 14

for t in tqdm(range(0, 100), position=1, desc="n_iteration", leave=False):
    context_learners = ecomm7.get_context_tree().get_leaves()
    idxs = []
    pulled_arms = np.zeros(shape=(NUM_OF_USERS_CLASSES, NUM_OF_PRODUCTS))

    if t % split_time == 0 and t!=0:
        for learner in context_learners:
            learner.evaluate_splitting_condition(features,
                                                 ecomm7.get_pulled_arms()[-split_time :],
                                                 ecomm7.get_collected_rewards()[-split_time :],
                                                 ecomm7.get_collected_sold_items()[-split_time :])
        
        context_learners = ecomm7.get_context_tree().get_leaves()

    for learner in context_learners:
        arm, idx = learner.pull_arm()
        idxs.append(idx)
        pulled_arms[idx, :] = arm

    reward, estimated_sold_items = env.round_step7(
        pulled_arms, B_cap, nodes_activation_probabilities, num_sold_items)

    for i in range(len(context_learners)):
        context_learners[i].update(
            pulled_arms[idxs[i][0]],
            np.sum(reward[idxs[i]], axis=0),
            np.sum(estimated_sold_items[idxs[i]], axis=0)
        )


    ecomm7.update_history(pulled_arms, reward, estimated_sold_items)


No better split found!


No better split found!


No better split found!


No better split found!


Better split_found {'B': 1} {'A': 0, 'C': 2, 'D': 0}


AssertionError: 

In [17]:
idxs

[[1], [0, 2]]

In [18]:
pulled_arms[idxs[0]].shape

(1, 5)